# Chapter 5: Text Classification

In [39]:
import spacy
import torchtext
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data
import torch.nn.functional as F
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading & Data Cleaning

In [3]:
device = "cuda"

In [4]:
# You'll probably need to use the 'python' engine to load the CSV
# tweetsDF = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None)
tweetsDF = pd.read_csv("/content/drive/MyDrive/Datasets/Sentiment140 dataset/training.1600000.processed.noemoticon.csv",
                       engine="python", header=None)

In [5]:
tweetsDF

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [6]:
tweetsDF[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

In [7]:
# Создаем отдельный столбец с метками
tweetsDF["sentiment_cat"] = tweetsDF[0].astype('category')

In [8]:
# Кодируем метки в числа в отдельном столбце
tweetsDF["sentiment"] = tweetsDF["sentiment_cat"].cat.codes

In [9]:
tweetsDF[100:110]

,0,1,2,3,4,5,sentiment_cat,sentiment
100,0,1467836873,Mon Apr 06 22:26:33 PDT 2009,NO_QUERY,LeakySpoon,Body Of Missing Northern Calif. Girl Found: P...,0,0
101,0,1467837189,Mon Apr 06 22:26:38 PDT 2009,NO_QUERY,cityrat59,@mangaaa I hope they will increase the capacit...,0,0
102,0,1467837384,Mon Apr 06 22:26:42 PDT 2009,NO_QUERY,kenandise,Behind on my classes for work,0,0
103,0,1467837470,Mon Apr 06 22:26:43 PDT 2009,NO_QUERY,annette414,watching &quot;House&quot;,0,0
104,0,1467837579,Mon Apr 06 22:26:45 PDT 2009,NO_QUERY,blackheartbunny,@kpreyes Remember my bum leg? Strikes back thi...,0,0
105,0,1467837602,Mon Apr 06 22:26:45 PDT 2009,NO_QUERY,GetGary,"@paradisej cool, i will. their are all kinds o...",0,0
106,0,1467837762,Mon Apr 06 22:26:48 PDT 2009,NO_QUERY,Dogbook,Emily will be glad when Mommy is done training...,0,0
107,0,1467837819,Mon Apr 06 22:26:49 PDT 2009,NO_QUERY,SimoniNicole,would rather the first party send bad messages...,0,0
108,0,1467837888,Mon Apr 06 22:26:52 PDT 2009,NO_QUERY,wireandroses,@Henkuyinepu it's overrated,0,0
109,0,1467838188,Mon Apr 06 22:26:54 PDT 2009,NO_QUERY,jess_higley,@marykatherine_q i know! I heard it this after...,0,0


In [10]:
# Перемешивание строк в датафрейме (frac=1 отражает в переменную весь датафрейм; frac=0.5 - выборка половины датафрейма)
tweetsDF = tweetsDF.sample(frac=1)

In [11]:
tweetsDF

,0,1,2,3,4,5,sentiment_cat,sentiment
540163,0,2199657192,Tue Jun 16 17:49:03 PDT 2009,NO_QUERY,bes21,writting learning goals,0,0
1437687,4,2061214688,Sat Jun 06 20:24:39 PDT 2009,NO_QUERY,CathyAnne,@TheRealJordin They are both great. I'm glad ...,4,1
1288716,4,2002520916,Tue Jun 02 04:27:13 PDT 2009,NO_QUERY,abooth202,Got up at 9. Been in library doing some last r...,4,1
1478074,4,2066410171,Sun Jun 07 10:28:05 PDT 2009,NO_QUERY,brittneyskye69,@realmarycarey change your flight babe and com...,4,1
283291,0,1992863594,Mon Jun 01 09:26:45 PDT 2009,NO_QUERY,lizziedr2,@mitchelmusso Hope you will give me a reply!,0,0
...,...,...,...,...,...,...,...,...
230193,0,1978681155,Sun May 31 00:07:03 PDT 2009,NO_QUERY,ouroborosJB,@30SECONDSTOMARS had my entering exams and u g...,0,0
416521,0,2061323373,Sat Jun 06 20:37:49 PDT 2009,NO_QUERY,Ballinkyle94,She is preettay in a odd odd way but she is Be...,0,0
1043035,4,1957275732,Fri May 29 00:01:22 PDT 2009,NO_QUERY,butterflykisser,"@tugbucket Good, thought it was an underhand w...",4,1
343713,0,2015518508,Wed Jun 03 04:52:32 PDT 2009,NO_QUERY,gwenlister,I forgot my fregging iPod at home today! I hav...,0,0


In [12]:
# Сохраняем измененный датасет на диск
tweetsDF.to_csv("/content/drive/MyDrive/Datasets/Sentiment140 dataset/train-processed.csv", 
                header=None, index=None) 
# Сохраняем отдельно выборку случайных 10000 твитов
tweetsDF.sample(10000).to_csv("/content/drive/MyDrive/Datasets/Sentiment140 dataset/train-processed-sample.csv",
                              header=None, index=None)

# Определение полей

In [13]:
# Объект класса Field для меток (без токенизации)
LABEL = data.LabelField()     
# Объект класса Field для твитов с токенизацей (используется токенизатор spacy и приведение к нижнему регистру)                                                  
TWEET = data.Field('spacy', tokenizer_language='en_core_web_sm', lower=True)    # Можно использовать токенизацию 
                                                                                  # по пробелу (string.split по умолчанию) - быстрее, но хуже



После определения этих полей нам нужно создать список, который свяжет
их со списком строк в CSV:

In [14]:
# Список, связывающий столбцы с полями
fields = [('score',None), ('id',None), ('date',None), ('query',None),
          ('name',None), ('tweet', TWEET), ('category',None), ('label',LABEL)]

# **Create our Dataset and DataLoaders**

Вооружившись объявленными полями, мы используем TabularDataset,
чтобы применить это определение к CSV:

In [15]:
twitterDataset = data.dataset.TabularDataset(
        path="/content/drive/MyDrive/Datasets/Sentiment140 dataset/train-processed.csv", 
        format="CSV", 
        fields=fields,
        skip_header=False)

Наконец, мы можем разделить наборы данных на обучение, тестирование
и верификацию с помощью метода split():

In [16]:
(train, test, valid) = twitterDataset.split(split_ratio=[0.8,0.1,0.1],
                                            stratified=True, strata_field='label')

(len(train),len(test),len(valid))

(1280000, 160000, 160000)

In [17]:
vars(train.examples[50])

{'label': '0', 'tweet': ['is', 'hatin', 'this', 'rain']}

In [18]:
vars(train.examples[30])['label']

'0'

In [19]:
count_0 = 0
count_1 = 0
for i in range(len(train)):
    if vars(train.examples[i])['label']=='0':
        count_0+=1
    else:
        count_1+=1
print(f"train_0: {count_0}")
print(f"train_1: {count_1}")

count_0 = 0
count_1 = 0
for i in range(len(test)):
    if vars(test.examples[i])['label']=='0':
        count_0+=1
    else:
        count_1+=1
print(f"test_0: {count_0}")
print(f"test_1: {count_1}")

count_0 = 0
count_1 = 0
for i in range(len(valid)):
    if vars(valid.examples[i])['label']=='0':
        count_0+=1
    else:
        count_1+=1
print(f"valid_0: {count_0}")
print(f"valid_1: {count_1}")


train_0: 640000
train_1: 640000
test_0: 80000
test_1: 80000
valid_0: 80000
valid_1: 80000


# **Построение словаря**

Обычно это делается для того, чтобы предотвратить
создание огромной модели, требующей много памяти. В конце концов, мы
не хотим перегружать наши графические процессоры. Давайте ограничим
словарь в наборе данных для обучения максимумом в 20 000 слов.

In [20]:
vocab_size = 20000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)

In [21]:
len(TWEET.vocab)

20002

Да, мы указали 20 000 слов, но torchtext по умолчанию
добавит еще два специальных токена `<unk>` для неизвестных слов (например, тех, которые были отброшены в результате ограничения 20 000 слов,
заданного с помощью max_size) и `<pad>`, токен отступа, который будет
использован для подгонки всего текста примерно до одного размера,
чтобы обеспечить эффективное пакетирование на GPU (помните, что его
скорость зависит от работы на регулярных пакетах). Также можно задать
символы eos_token или init_token при объявлении поля, но они не включены по умолчанию.

In [22]:
TWEET.vocab.freqs.most_common(10)

[('i', 596282),
 ('to', 448140),
 ('the', 415338),
 ('a', 301161),
 ('my', 250575),
 ('and', 236673),
 ('you', 190264),
 ('is', 185184),
 ('for', 171152),
 ('in', 167678)]

Практически то, что вы и ожидали, поскольку мы не удаляем стоп-слова
с помощью токенизатора spaCy. (Поскольку это всего 140 символов, если
мы это сделаем, то наша модель может потерять слишком много информации).

Мы почти закончили с наборами данных. Теперь нужно создать загрузчик данных для подачи в цикл обучения. torchtext предоставляет метод
BucketIterator, который будет производить то, что он называет Batch, —
это немного похоже на загрузчик данных, который мы использовали на
изображениях.

(Вскоре вы увидите, что нужно обновить цикл обучения, чтобы справиться
с некоторыми странностями интерфейса Batch.)

In [23]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_size = 32,
    device = device,
    sort_key = lambda x: len(x.tweet),
    sort_within_batch = False)

# **Our First LSTM**

# Создание модели

In [24]:
class OurFirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(OurFirstLSTM, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim,  
                hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 2)

    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = OurFirstLSTM(100,300, 20002)
model.to(device)

OurFirstLSTM(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

In [25]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):
     
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [26]:
train(10, model, optimizer, criterion, train_iterator, valid_iterator)

Epoch: 1, Training Loss: 18.51, Validation Loss: 11.27
Epoch: 2, Training Loss: 18.33, Validation Loss: 10.55
Epoch: 3, Training Loss: 17.98, Validation Loss: 10.88
Epoch: 4, Training Loss: 17.88, Validation Loss: 10.44
Epoch: 5, Training Loss: 17.67, Validation Loss: 11.07
Epoch: 6, Training Loss: 17.51, Validation Loss: 12.05
Epoch: 7, Training Loss: 17.38, Validation Loss: 10.83
Epoch: 8, Training Loss: 17.30, Validation Loss: 10.24
Epoch: 9, Training Loss: 17.24, Validation Loss: 10.49
Epoch: 10, Training Loss: 17.26, Validation Loss: 10.50


In [42]:
# Проверка на тестовом наборе

num_correct = 0 
num_examples = 0
valid_loss = 0.0
model.eval()
for batch_idx,batch in enumerate(test_iterator):
    predict = model(batch.tweet)
    loss = criterion(predict,batch.label)
    valid_loss += loss.data.item() * batch.tweet.size(0)
    correct = torch.eq(torch.max(F.softmax(predict), dim=1)[1], batch.label).view(-1)
    num_correct += torch.sum(correct).item()
    num_examples += correct.shape[0]
valid_loss /= len(test_iterator)
print('test_loss: {:.2f}, accuracy = {:.2f}'.format(valid_loss, num_correct / num_examples))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


test_loss: 10.50, accuracy = 0.58


# Making predictions

In [28]:
def classify_tweet(tweet):
    categories = {0: "Negative", 1:"Positive"}
    processed = TWEET.process([TWEET.preprocess(tweet)])
    processed = processed.to(device)
    model.eval()
    return categories[model(processed).argmax().item()]

In [49]:
classify_tweet(str(test[99]))

'Negative'

In [50]:
vars(test.examples[99])['label']

'0'

# **Data Augmentation**

# Случайное удаление

In [26]:
def random_deletion(words, p=0.5):
    if len(words) == 1:
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    if len(remaining) == 0:
        return [random.choice(words)]
    else:
        return remaining

# **Случайная перестановка**

In [26]:
def random_swap(sentence, n=5):
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return sentence

# Случайная вставка

In [26]:
# Note: you'll have to define remove_stopwords() and get_synonyms() elsewhere

def random_insertion(sentence,n):
    words = remove_stopwords(sentence)
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym)
    return sentence

# **Обратный перевод**

In [52]:
pip install googletrans

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 1.3 MB 10.4 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15734 sha256=40a3755f0dcd4512be2451e302f420326324205d4a7bc58f08630bb843c5e16f
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans


In [58]:
import googletrans
import random

translator = googletrans.Translator()

sentences = ['The cat sat on the mat']

translations_fr = translator.translate(sentences, dest='fr')
fr_text = [t.text for t in translations_fr] 
translations_en = translator.translate(fr_text, dest='en')
en_text = [t.text for t in translations_en]
print(en_text)   

available_langs = list(googletrans.LANGUAGES.keys())
tr_lang = random.choice(available_langs)
print(f"Translating to {googletrans.LANGUAGES[tr_lang]}")

translations = translator.translate(sentences, dest=tr_lang)
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=tr_lang, dest='en')
en_text = [t.text for t in translations_en_random]
print(en_text)

AttributeError: ignored

In [59]:
import random
available_langs = list(googletrans.LANGUAGES.keys())
tr_lang = random.choice(available_langs)
print(f"Translating to {googletrans.LANGUAGES[tr_lang]}")
translations = translator.translate(sentences, dest=tr_lang)
t_text = [t.text for t in translations]
print(t_text)
translations_en_random = translator.translate(t_text, src=tr_lang, dest='en')
en_text = [t.text for t in translations_en_random]
print(en_text)

Translating to latvian


AttributeError: ignored